LSTM MODEL

In [ ]:
# imports
import datetime
from datetime import date
import pandas as pd
import numpy as np
from plotly import __version__
%matplotlib inline

import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.offline import iplot

import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 


cf.go_offline()

Uploading CSV file

In [ ]:
# File named in ./data.csv
# NOTE: Please "Allow 3rd Party Cookies" in Chrome Options

from google.colab import files
uploaded = files.upload()

In [ ]:
print(uploaded)

In [ ]:
import pandas as pd
import io
  
df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

print(df)

print(df[["RF", "MaxT"]])

print(df['RF'][0])

print(type(df['RF']))

print(type(df['RF'].to_numpy()))

Visualizing Data

In [ ]:
# Creating Simple Dynamic Graph to see all data
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

fig = plt.figure(figsize=(23, 6), dpi=80)
ax = plt.axes()

# NOTE : Chane the value of var for any other header to get different results
var = "RF"
plt.plot(df['Date'].to_numpy(), df[var].to_numpy());

In [ ]:
init_notebook_mode(connected=False)

In [ ]:
# Create function for Colab
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
  '''))

In [ ]:
configure_plotly_browser_state()
df.iplot(kind='line',x='Date',y=['RF'], color=['white'], 
theme='solar', mode='markers',title='Annual Rainfall in the city Peachtree City, GA')
plt.show()

In [ ]:
configure_plotly_browser_state()
df.iplot(kind='line',x='Date',y=['MinT', "MaxT"], color=['white', 'gold'], 
theme='solar', mode='markers',title='Annual Rainfall in the city Peachtree City, GA')
plt.show()

Pre Processing Data

In [ ]:
# preproceessing data
def one_hot_prob_dist(val):
  if val == 0 :
    return [1, 0, 0, 0]
  elif val == 1 :
    return [0, 1, 0, 0]
  elif val == 2 :
    return [0, 0, 1, 0]
  elif val == 3 :
    return [0, 0, 0, 1]
  else :
    print(val)
    raise ValueError

def prepare_data(data, n_features, timesteps):
  # Dividing X and y
  X = data[["RF", "MaxT", "MinT", "RH-I", "RH-II", "C2", "SS", "WD1", "WD2", "WS"]]
  y_temp = data["Disease"]
  y = []

  # print("X & y : ", "\n", X, "\n", y_temp)

  # Create Numpy arrays
  X = X.to_numpy()
  y_temp = y_temp.to_numpy()

  for i in range(len(y_temp)):
    arr = one_hot_prob_dist(y_temp[i])
    # print(arr)
    # np_arr = np.array([arr])
    # np.append(y, np_arr, axis=0)
    y.append(arr)

  y = np.array(y)

  # print("X & y (in numpy) : ", "\n", X, "\n", y)
  # print("X & y (shape) : ", X.shape, ", ", y.shape)

  # Normalizing values
  # X = (X - X.min(0)) / X.ptp(0)
  # y = (y - y.min(0)) / y.ptp(0)

  # print("X & y (normalized) : ", "\n", X, "\n", y)

  # reshaping data into 3D structure [example, timesteps, features]
  X = X.reshape(X.shape[0], timesteps, X.shape[1])
  # y = y.reshape(y.shape[0], timesteps, y.shape[1])
  
  return X, y

def split_data(X, y, ratio=0.98):
  # Create X_test, X_train, y_test, y_train
  if ratio > 1:
    raise Error
  else :
    tot = X.shape[0]
    div = round(tot*ratio)

    # splitting
    X_train = X[:div, :, :]
    y_train = y[:div]

    X_test = X[div:, :, :]
    y_test = y[div:]
  
  return X_train, y_train, X_test, y_test

In [ ]:
# defining data
n_features = 10 
timesteps = 1

X, y = prepare_data(df, n_features, timesteps)

In [ ]:
X_train, y_train, X_test, y_test = split_data(X, y, ratio=0.8)

In [ ]:
print("X_train : ", X_train[:10], "\n\ny_test : ", y_train[:10])
print("\n\nX_train and y_test (shape) : ", X_train.shape, ", ", y_test.shape)

Training LSTM Model

In [ ]:
# impots
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
# defining model
model = Sequential()
model.add(LSTM(1, activation='relu', input_shape=(timesteps, n_features)))
# model.add(LSTM(64, activation='relu', return_sequences=True))
# model.add(LSTM(64, activation='relu'))

model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

# compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# model sumary
print(model.summary())

# train model
history = model.fit(X_train, y_train, epochs=50, verbose=1)

# validation_data=(X_test, y_test), batch_size=20,

In [ ]:
# check for overfitting and underfitting
print(history.history['loss'])
print(history.history['accuracy'])

Predicting Data and Seeing results using Test Set 

In [ ]:
def upscale(value, range=3):
  return round(value*range)

print(X_test[0])

In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:3])
print("predictions shape:", predictions.shape)

In [ ]:
print(predictions)

Visualizing Ouput